In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from scipy.optimize import curve_fit

In [2]:
def getfiles(filepath):
    dfs = []
    for i in (glob.glob(filepath)):
        df = pd.read_csv(i, header=0)
        dfs.append(df)
    return pd.concat(dfs, axis=0, ignore_index=True).drop(columns='Unnamed: 0')

### Question 2

In [3]:
#get trade data for SH600519
df_t19 = getfiles('SH600519/trade*.csv')

Calibrate $\lambda$ by grouping trades into 5 minute intervals and getting the average frequency of number of trades per interval 

In [4]:
#convert date and time to datetime object
df_t19['timestamp'] = pd.to_datetime((df_t19['date'].astype(str) + df_t19['time'].astype(str)), format= '%Y%m%d%H%M%f')
#group and get distribution of number of trades over the time period
group = df_t19.groupby(pd.Grouper(key='timestamp', freq='5Min')).count()['ntrade']
num_trades = group.value_counts() / group.shape[0]
#plot and get variables
#num_trades[1:].sort_index().plot(figsize=[10,5])
lambda_tau = group[group > 0].mean()
print('lambda_tau: ' , lambda_tau )
print('lambda: ', lambda_tau/5 )

lambda_tau:  235.41086040168608
lambda:  47.082172080337216


Calibrate $\beta$ by normalizing the volumes and taking the reciprical of the average normalized volumes

In [5]:
nml = df_t19['ntrade'] / df_t19['ntrade'].mean()
size = nml.value_counts() / nml.sum()
#size.sort_index().cumsum().plot(figsize=[10,5])
print('beta: ', 1/nml.mean())

beta:  0.9999999999999998


Calibrate p by obtaining the probability of a buy or a sell based on column BS

In [6]:
pd.DataFrame(df_t19['BS'].value_counts()/df_t19.shape[0]).T

,S,B,
BS,0.507318,0.491282,0.0014


Probability of a Buy is 49.1% and of a Sell is 50.7%

Calibrate $\rho$ as equal to the pearson correlation of the sign B or S

In [7]:
df_t19['sign'] = np.where(df_t19['BS'] == 'B', 1, np.where(df_t19['BS'] == 'S', -1, 0))

In [8]:
print('rho: ', df_t19['sign'].autocorr().round(3))

rho:  0.561


### Question 3

In [9]:
df_t19 = getfiles('SH600519/trade*.csv')
df_t98 = getfiles('SH601398/trade*.csv')
df_q19 = getfiles('SH600519/quote*.csv')
df_q98 = getfiles('SH601398/quote*.csv')

Predict the sign by first calculating the midquote as average of bid and ask prices. Then merge the data on timestamp and price. Make prediction based on if Midprice is over or under the actual price

In [10]:
def predict_sign(trade_df, quote_df):
    #make timestamps
    trade_df['timestamp'] = pd.to_datetime((trade_df['date'].astype(str) + trade_df['time'].astype(str).str.zfill(4)), format= '%Y%m%d%H%M%f')
    quote_df['timestamp'] = pd.to_datetime((quote_df['date'].astype(str) + quote_df['time'].astype(str).str.zfill(4)), format= '%Y%m%d%H%M%f')
    #calc midquote
    quote_df['Midquote'] = (quote_df['AskPrice1'] + quote_df['BidPrice1']) / 2
    
    #merge on price and timestamp
    merged_df = pd.merge(trade_df[['timestamp', 'price', 'sign', 'ntrade', 'BS', 'date', 'time']], quote_df[['timestamp', 'price', 'Midquote']], on=['timestamp','price'])
    #make the prediction
    merged_df['sign'] = np.where(merged_df['price'] < merged_df['Midquote'], -1, 
                                 np.where(merged_df['price'] > merged_df['Midquote'], 1, 0))
    #get accuracy
    merged_df['Pred_Acc'] = np.where(((merged_df['sign'] == -1) & (merged_df['BS'] == 'S')) | ((merged_df['sign'] == 1) & (merged_df['BS'] == 'B')), 1, 0)
    
    return merged_df

In [11]:
merged_19 = predict_sign(df_t19, df_q19)
merged_98 = predict_sign(df_t98, df_q98)

In [12]:
print('Accuracy of prediction for SH600519: ', '{:.2%}'.format(merged_19['Pred_Acc'].sum()/merged_19['Pred_Acc'].count()))
print('Accuracy of prediction for SH601398: ', '{:.2%}'.format(merged_98['Pred_Acc'].sum()/merged_98['Pred_Acc'].count()))
print('Overall accuracy: ', '{:.2%}'.format((merged_19['Pred_Acc'].sum() + merged_98['Pred_Acc'].sum()) / (merged_19['Pred_Acc'].count() + merged_98['Pred_Acc'].count())))

Accuracy of prediction for SH600519:  91.65%
Accuracy of prediction for SH601398:  97.05%
Overall accuracy:  94.59%


### Question 4

Create a function to take in different intervals of time $i$ and calculate the IMB based on the midquote function and final datasets above. The data is then filtered by its sign (1, -1) to calculate the positive and negative parameters. Gamma is derived from fitting a curve to the equation and taking the optimal value to minimize the sum of squared residuals

In [13]:
def betagamma(i, df):
    new_df = df.copy()
    new_df['time'] = new_df['time'].astype(str)
    new_df['time'] = pd.to_numeric(new_df['time'].str[:-5])
    new_df['nml'] = new_df['ntrade']/ new_df['ntrade'].mean()
    new_df['IMB'] = new_df['sign'] * new_df['nml']
    
    ret = new_df[['date','time','Midquote', 'timestamp']]
    ret = ret.groupby(['date',pd.cut(ret['time'], np.arange(925, 1500+i, i),right=False, include_lowest=True)]).Midquote.apply(lambda x:(x.tail(1).values[0]-x.head(1).values[0])/x.head(1).values[0]).dropna()
    
    size = new_df[['date', 'time', 'IMB', 'nml']]
    size = size.groupby(['date',pd.cut(size['time'], np.arange(925, 1500+i, i), right=False, include_lowest=True)]).agg({'IMB': 'sum', 'nml':['size', 'sum']}).dropna()
    
    df_concat = pd.concat([size, ret], axis=1)
    df_concat.columns = ['IMB', 'Size', 'Sum', 'Ret']
    df_concat['Sign'] = np.where(df_concat['IMB'] > 0, 1, -1)
    stdev = df_concat['Ret'].std()
    
    pos_beta = 1 / (df_concat[df_concat['Sign'] == 1]['Sum'].sum() / df_concat[df_concat['Sign'] == 1]['Size'].sum())
    neg_beta = 1 / (df_concat[df_concat['Sign'] == -1]['Sum'].sum() / df_concat[df_concat['Sign'] == -1]['Size'].sum())
    
    def f_pos(IMB,gamma):
        return (pos_beta*stdev*(abs(IMB)**gamma))
    
    def f_neg(IMB,gamma):
        return (neg_beta*stdev*(abs(IMB)**gamma))
    
    df_concat.dropna(inplace=True)
    pos_gamma = curve_fit(f_pos, df_concat[df_concat['Sign'] == 1]['IMB'], df_concat[df_concat['Sign'] == 1]['Ret'])
    neg_gamma = curve_fit(f_neg, df_concat[df_concat['Sign'] == -1]['IMB'], df_concat[df_concat['Sign'] == -1]['Ret'])
    
    return pos_beta, neg_beta, pos_gamma[0], neg_gamma[0]

In [14]:
dfs = [merged_19, merged_98]
outdfs = []
for df in dfs:
    pbs = []
    nbs = []
    pgs = []
    ngs = []
    for i in range(1,6):
        pb, nb, pg, ng = betagamma(i, df)
        pbs.append(pb)
        nbs.append(nb)
        pgs.append(pg[0])
        ngs.append(ng[0])
    outdf = pd.DataFrame({'Positive Betas' : pbs, 'Negative Betas' : nbs, 'Positive Gammas' : pgs, 'Negative Gammas' : ngs})
    outdf.index.rename('minutes', inplace=True)
    outdf.index = outdf.index + 1
    outdfs.append(outdf)

Output for file SH600519

In [15]:
outdfs[0]

,Positive Betas,Negative Betas,Positive Gammas,Negative Gammas
minutes,,,,
1,0.980544,1.020623,-0.041744,1.179594e-08
2,0.982978,1.018402,-0.059119,9.460007e-09
3,0.986844,1.013896,-0.053463,2.645501e-08
4,0.989755,1.011497,-0.044441,1.397018e-08
5,0.984531,1.018049,-0.042368,1.773642e-09


Output for file SH601398

In [16]:
outdfs[1]

,Positive Betas,Negative Betas,Positive Gammas,Negative Gammas
minutes,,,,
1,0.977000,1.020249,-0.025925,1.448812e-08
2,0.984426,1.013759,-0.062777,4.609944e-09
3,0.978098,1.019258,-0.035577,5.062150e-09
4,0.982108,1.015885,-0.042953,3.170110e-08
5,0.984436,1.014209,-0.041945,3.059483e-08
